In [1]:
import sys
sys.path.append("./function_scripts")
import data_cleaning_fns as dcfns

import pandas as pd
import re
import pickle
import datetime as dt

#### Import 'dirty' kickstarter dataframes

In [2]:
dirty_pickle_files = ['all_data_dirty_1_1999.pickle',
                      'all_data_dirty_2000_2999.pickle',
                      'all_data_dirty_3000_3751.pickle']
with open(dirty_pickle_files[0], 'rb') as f:
    project_df_0 = pickle.load(f)
print(project_df_0.shape)
with open(dirty_pickle_files[1], 'rb') as f:
    project_df_1 = pickle.load(f)
print(project_df_1.shape)
with open(dirty_pickle_files[2], 'rb') as f:
    project_df_2 = pickle.load(f)
print(project_df_2.shape)

project_df = pd.concat([project_df_0, project_df_1, project_df_2])
print(project_df.shape)

(29740, 11)
(14901, 11)
(11237, 11)
(55878, 11)


In [3]:
project_df.reset_index(inplace=True, drop=True)

In [4]:
project_df.sample(2)

,kickstarter_page,funding,campaign_dates,backers,category,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,currency
48527,/projects/1553937149/gymnation-bringing-the-mu...,"\n\t\t\t\t\tFunding: 20 of 15,000 (",\n\t\t\t\t\tCampaign Dates: February 20th -> A...,\n\t\t\t\t\tBackers: 2,Games,622,7,10,8000,2681.582688,AU
13875,/projects/patriceregnier/the-terp-app-a-whole-...,"\n\t\t\t\t\tFunding: 7,650 of 21,000 (",\n\t\t\t\t\tCampaign Dates: August 20th -> Oct...,\n\t\t\t\t\tBackers: 67,Games,576,14,10,5000,1880.813064,$


#### Extract numerical data from text fields

###### Initialize columns for cleaned data with base values

In [5]:
project_df['funding_actual'] = 0
project_df['funding_goal'] = 0
project_df['n_backers'] = 0
project_df['start_date'] = 0
project_df['duration'] = 0

###### Apply function to extract data from text

In [6]:
project_df = project_df.apply(dcfns.data_from_text, axis=1)
project_df.head(2)

,kickstarter_page,funding,campaign_dates,backers,category,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,currency,funding_actual,funding_goal,n_backers,start_date,duration
0,/projects/1063028485/good-art-guide/,"\n\t\t\t\t\tFunding: £0 of £10,000 (",\n\t\t\t\t\tCampaign Dates: November 26th -> J...,\n\t\t\t\t\tBackers: 0,Art,93,1,5,5,0.00000,£,0,10000,0,2014-11-26,60
1,/projects/spiderwebcitytx/katlynn-maries-horro...,"\n\t\t\t\t\tFunding: 17 of 15,000 (",\n\t\t\t\t\tCampaign Dates: November 24th -> J...,\n\t\t\t\t\tBackers: 3,Art,106,3,2,100,45.50702,$,17,15000,3,2014-11-24,60


###### Drop now-unnecessary text columns

In [7]:
project_df.drop(columns=['funding', 'campaign_dates', 'backers'], inplace=True)
project_df.columns

Index(['kickstarter_page', 'category', 'description_length', 'n_pledges',
       'pledge_level_min', 'pledge_level_max', 'pledge_level_stddev',
       'currency', 'funding_actual', 'funding_goal', 'n_backers', 'start_date',
       'duration'],
      dtype='object')

###### Save intermediate version of data

In [8]:
with open('all_data_no_currency_conversion.pickle', 'wb') as f:
    pickle.dump(project_df,f)

#### Standardize Currencies

In [9]:
project_df.currency.value_counts()

$       42249
£        6861
CA       3365
AU       1807
€         601
US        599
NZ        326
SEK        26
NOK        22
DKK        22
Name: currency, dtype: int64

###### After some spot-checking, I am pretty confident that "US" and "$" both refer to US currency. Let's set both to "USD". Furthermore, let's convert all our currency tags to those used by the European Central Bank. Furthermore, let's limit our observations to those in USD, GBP, CAD, AUD, EUR and NZD.

In [10]:
project_df.currency = project_df.currency.apply(lambda x: 'USD' if x=='$' or x=='US '
                                                else ('GBP' if x=='£' else 
                                                      ('CAD' if x=='CA ' else
                                                       ('AUD' if x=='AU ' else
                                                       ('EUR' if x=='€' else
                                                       ('NZD' if x=='NZ ' else x))))))

In [11]:
project_df.currency.value_counts()

USD     42848
GBP      6861
CAD      3365
AUD      1807
EUR       601
NZD       326
SEK        26
NOK        22
DKK        22
Name: currency, dtype: int64

###### Drop unwanted currencies

In [12]:
wanted_currency_mask = project_df.currency.isin(['USD', 'GBP', 'CAD', 'AUD', 'EUR', 'NZD'])
project_df = project_df[wanted_currency_mask]
project_df.currency.value_counts()
#unwanted_currency_mask

USD    42848
GBP     6861
CAD     3365
AUD     1807
EUR      601
NZD      326
Name: currency, dtype: int64

###### Convert currencies using python currency_converter with data from European Central Bank 

In [13]:
project_df.shape

(55808, 13)

In [14]:
project_df = project_df.apply(dcfns.currency_change, axis=1)
project_df.drop(columns=['currency'], inplace=True)
project_df.head()

,kickstarter_page,category,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,funding_actual,funding_goal,n_backers,start_date,duration
0,/projects/1063028485/good-art-guide/,Art,93,1,7.886086,7.886086,0.000000,0.0,15772.172704,0,2014-11-26,60
1,/projects/spiderwebcitytx/katlynn-maries-horro...,Art,106,3,2.000000,100.000000,45.507020,17.0,15000.000000,3,2014-11-24,60
2,/projects/mnassar/wwwparkinglocatorcomau/,Technology,46,1,8.712913,8.712913,0.000000,0.0,43564.564775,0,2014-11-21,60
3,/projects/1934089488/not-enough-fish/,Film & Video,160,7,7.837804,783.780404,299.100883,0.0,1567.560808,0,2014-11-19,59
4,/projects/timodwyer/hurt-fate-feature-film/,Film & Video,153,10,1.564151,7820.753536,2342.290924,0.0,93849.042433,0,2014-11-17,60


In [15]:
with open('all_data_clean_original.pickle', 'wb') as f:
    pickle.dump(project_df,f)

##### Get dummies from category
###### Note that "Art" is dropped, so projects with all 0s in category columns are Art projects

In [34]:
dummy_columns = pd.get_dummies(project_df.category, drop_first=True)

In [35]:
df_full =pd.concat([project_df,dummy_columns], axis=1)
df_full.drop(columns=['category'], inplace=True)

In [36]:
df_full.sample(2)

,kickstarter_page,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,funding_actual,funding_goal,n_backers,start_date,...,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Sculpture,Technology,Theater
13556,/projects/593066327/my-very-first-ever-art-show/,98,2,40.0,80.0,20.000000,721.0,700.0,6,2014-09-03,...,0,0,0,0,0,0,0,0,0,0
11517,/projects/callmemabey/peace-love-happiness/,159,5,5.0,100.0,34.727511,0.0,12500.0,0,2014-09-12,...,0,0,0,0,0,0,0,0,0,0


#### Add in funding % and get rid of columns we will not use in our models

In [37]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55808 entries, 0 to 55877
Data columns (total 26 columns):
kickstarter_page       55808 non-null object
description_length     55808 non-null int64
n_pledges              55808 non-null int64
pledge_level_min       55808 non-null float64
pledge_level_max       55808 non-null float64
pledge_level_stddev    55808 non-null float64
funding_actual         55808 non-null float64
funding_goal           55808 non-null float64
n_backers              55808 non-null object
start_date             55808 non-null object
duration               55808 non-null int64
Comics                 55808 non-null uint8
Crafts                 55808 non-null uint8
Dance                  55808 non-null uint8
Design                 55808 non-null uint8
Fashion                55808 non-null uint8
Film & Video           55808 non-null uint8
Food                   55808 non-null uint8
Games                  55808 non-null uint8
Journalism             55808 non-null uint

In [38]:
df_to_model = df_full
df_to_model.drop(columns=['kickstarter_page', 'start_date', 'n_backers'], inplace=True)

In [39]:
df_to_model['funding_percent'] = df_to_model.funding_actual / df_to_model.funding_goal
df_to_model.drop(columns=['funding_actual'], inplace=True)
df_to_model.head()

,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,funding_goal,duration,Comics,Crafts,Dance,...,Food,Games,Journalism,Music,Photography,Publishing,Sculpture,Technology,Theater,funding_percent
0,93,1,7.886086,7.886086,0.000000,15772.172704,60,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,106,3,2.000000,100.000000,45.507020,15000.000000,60,0,0,0,...,0,0,0,0,0,0,0,0,0,0.001133
2,46,1,8.712913,8.712913,0.000000,43564.564775,60,0,0,0,...,0,0,0,0,0,0,0,1,0,0.000000
3,160,7,7.837804,783.780404,299.100883,1567.560808,59,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,153,10,1.564151,7820.753536,2342.290924,93849.042433,60,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [40]:
with open('all_data_to_model.pickle', 'wb') as f:
    pickle.dump(df_full,f)